## Analisis de datos

#### Creamos un dataframe con los datos de general data y lo fragmentamos y guardamos en un csv por departamento

In [1]:
import os
import pandas as pd

In [14]:
# Carga datos en un DataFrame
general_data = pd.read_csv('Reporte_Capturas_Polic_a_Nacional_20241025.csv', low_memory=False)
general_data =  general_data[~general_data["DEPARTAMENTO"].isin(["QUITO", "MADRID","NEW YORK","NEW JERSEY","FLORIDA"])]

def csv_by_filter(data, filter_column, output_folder, out = True):
    # Especifica la carpeta donde guardar los archivos CSV
    os.makedirs(output_folder, exist_ok=True)  # Crea la carpeta si no existe

    # Filtra y guarda un CSV para cada departamento
    for filter_column, data in general_data.groupby('DEPARTAMENTO'):
        # Genera el nombre del archivo para cada departamento
        output_file = os.path.join(output_folder, f"{filter_column}.csv")
        if (out):
            print(f"Guardando {output_file}... con {len(data)} filas")
        # Guarda el DataFrame filtrado como CSV
        data.to_csv(output_file, index=False)
    if (out):
        print("Archivos CSV generados en la carpeta:", output_folder)

# Filtra y guarda un CSV para cada departamento
csv_by_filter(general_data, 'departamento', 'departamentos', out = False)

In [19]:
cauca = pd.read_csv('departamentos/CAUCA.csv', low_memory=False)

In [18]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Carga los datos
cauca = pd.read_csv('departamentos/CAUCA.csv', low_memory=False)

# Asegúrate de que las columnas sean de tipo string
cauca[["DEPARTAMENTO", "MUNICIPIO", "CODIGO_DANE", "GENERO", "DESCRIPCION_CONDUCTA_CAPTURA", "GRUPO_ETARIO"]] = cauca[["DEPARTAMENTO", "MUNICIPIO", "CODIGO_DANE", "GENERO", "DESCRIPCION_CONDUCTA_CAPTURA", "GRUPO_ETARIO"]].astype(str)

# Codificación One-Hot
codificador = OneHotEncoder(sparse_output=False)
codificacion = codificador.fit_transform(cauca[["DEPARTAMENTO", "MUNICIPIO", "CODIGO_DANE", "GENERO", "DESCRIPCION_CONDUCTA_CAPTURA", "GRUPO_ETARIO"]])

# Convierte la columna FECHA_HECHO a formato datetime
cauca['FECHA_HECHO'] = pd.to_datetime(cauca['FECHA_HECHO'], format="%d/%m/%Y")

# Crear nombres de columnas para el DataFrame resultante
nombres_columnas = []
for i, cat in enumerate(codificador.categories_):
    nombres_columnas.extend([f"{codificador.feature_names_in_[i]}_{val}" for val in cat])

# Crear el DataFrame con los resultados de la codificación
nueva_data = pd.DataFrame(codificacion, columns=nombres_columnas)

# Unir los dos DataFrames
cauca_completo = pd.concat([cauca.reset_index(drop=True), nueva_data.reset_index(drop=True)], axis=1)

# Guardar el DataFrame combinado en un archivo CSV
output_file = 'cauca_completo.csv'
cauca_completo.to_csv(output_file, index=False)

print(f"El DataFrame combinado se ha guardado en: {output_file}")

El DataFrame combinado se ha guardado en: cauca_completo.csv
